# *IceFlow & icepyx*: Altimetry Time Series Tutorial
### NASA Earthdata Webinar - April 2021</b>

This tutorial demonstrates how to harmonize several NASA altimetry data sets with varying temporal coverage, formats, and coordinate reference frames using the IceFlow and icepyx Python tools. Please refer to the 0_introduction.ipynb notebook for detailed information on IceFlow and the data sets you will be exploring in this tutorial. 

#### Objectives:
1. Use the IceFlow map widget to select and visualize an area of interest.
2. Access coincident ICESat/Glas, Operation IceBridge, and ICESat-2 data over the same spatial region.
3. Use the community-developed icepyx python library to subset ICESat-2 data.
4. Learn about advanced icepyx capabilities including data value visualization prior to download. 
5. Extract common data variables into a Geopandas dataframe.
7. Plot and visualize the altimetry time series to detect glacial elevation change over time.

<b>Authors:</b><br />
<span style="font-size:larger;">Jessica Scheick</span>, *University of New Hampshire*, Durham, New Hampshire<br />
<span style="font-size:larger;">Nicholas Kotlinski & Amy Steiker</span>, *NASA National Snow and Ice Data Center DAAC*, Boulder, Colorado, USA

---

#### Running this tutorial locally

To run this notebook locally, you must first set up your computing environment. Please see the [repository readme](https://github.com/nsidc/NSIDC-Data-Tutorials#usage-with-binder) for instructions on several ways (using Binder, Docker, or Conda) to do this.

## 1. NASA's Earthdata Credentials

To access data using the *IceFlow* library and *icepyx* package, it is necessary to log into [Earthdata Login](https://urs.earthdata.nasa.gov/). To do this, enter your NASA Earthdata credentials in the next step after executing the following code cell.

**Note**: If you don't have NASA Earthdata credentials you will need to register first at the link above. An account is free and available to everyone!

In [ ]:
from iceflow.ui import IceFlowUI # Importing IceFlow client library
client = IceFlowUI()
client.display_credentials()

In [ ]:
# This cell will verify if your credentials are valid. 
# This may take a little while. If it fails, please try again.

authorized = client.authenticate()
if authorized is None:
    print('Earthdata Login not successful')
else:
    print('Earthdata Login successful!')

**Note:** If the output shows "Authenticated with NASA Earthdata", then you are ready to proceed!

---

## 2. Accessing and harmonizing data across missions

#### 2.1. Available data

The following table provides information on the data available using the IceFlow API.

|IceFlow Name | Data Set| Spatial Coverage | Temporal Coverage| Mission  | Sensors  |
|-------------|---------|------------------|------------------|----------|----------|
**ATM1B** |[BLATM L1B](https://nsidc.org/data/BLATM1B)| South: N:-53, S: -90, E:180, W:-180 <br> North: N:90, S: 60, E:180, W:-180 | 23 Jun. 1993 - 30 Oct. 2008 | Pre-IceBridge | ATM  | 
**ATM1B** |[ILATM L1B V1](https://nsidc.org/data/ILATM1B/versions/1) | South: N:-53, S: -90, E:180, W:-180 <br> North: N:90, S: 60, E:180, W:-180 | 31 Mar. 2009 - 8 Nov. 2012  <br> (updated 2013) | IceBridge | ATM | 
**ATM1B** |[ILATM L1B V2](https://nsidc.org/data/ILATM1B/versions/2)| South: N:-53, S: -90, E:180, W:-180 <br> North: N:90, S: 60, E:180, W:-180 | 20 Mar. 2013 - 16 May 2019  <br> (updated 2020)| IceBridge|ATM|
**ILVIS2** |[ILVIS2](https://nsidc.org/data/ILVIS2)| North: N:90, S: 60, E:180, W:-180|25 Aug. 2017 - 20 Sept. 2017|IceBridge | ALTIMETERS, LASERS, LVIS |
**GLAH06** |[GLAH06](https://nsidc.org/data/GLAH06/)| Global: N:86, S: -86, E:180, W:-180|20 Feb. 2003 - 11 Oct. 2009|ICESat/GLAS | ALTIMETERS, CD, GLAS, GPS, <br> GPS Receiver, LA, PC|
**ATL03** |[ATL03](https://nsidc.org/data/ATL03/)| Global: N:90, S: -90, E:180, W:-180|13 Oct. 2018 - Present|ICESat-2 | ATLAS |
**ATL06** |[ATL06](https://nsidc.org/data/ATL06/)| Global: N:90, S: -90, E:180, W:-180|13 Oct. 2018 - Present|ICESat-2 | ATLAS |
**ATL07** |[ATL07](https://nsidc.org/data/ATL07/)| Global: N:90, S: -90, E:180, W:-180|13 Oct. 2018 - Present|ICESat-2 | ATLAS |
**ATL08** |[ATL08](https://nsidc.org/data/ATL08/)| Global: N:90, S: -90, E:180, W:-180|13 Oct. 2018 - Present|ICESat-2 | ATLAS |

**Notes**:
* Due to the nature of the **ILVIS2** product, IceFlow doesn't provide a common data dictionary to be used in conjunction with the other data sets. Data are accessible through IceFlow, but you will need to harmonize the data to your own specifications.
* If you have questions about the data sets please refer to the user guides available at each link in the table, or contact NSIDC user services at nsidc@nsidc.org

--- 


#### 2.2. Choosing Corrections: Using the ITRF and Epoch values
* The differences between International Terrestrial Reference Frame (ITRF) corrections is negligible in most cases, and corrections should only be applied by users who are familiar with the procedures behind these corrections.

* The optional ***ITRF*** parameter allows you to choose an ITRF reference to which the data will be transformed via the published ITRF transformation parameters. A more detailed overview of the ITRF parameters can be found in [Applying Coordinate Transformations to Facilitate Data Comparison](corrections.ipynb). This parameter is optional but must be used if you want to specify an epoch. Available values are: **ITRF2000, ITRF2008, ITRF2014**</br>
Example: `'itrf': 'ITRF2014'`
* The ***epoch*** parameter is optional and entered in decimal years to which the data will be transformed via the ITRF Plate Motion Model corresponding to ITRF. This parameter can only be used if the ***ITRF*** parameter is specified and set to either 2008 or 2014, as only ITRF2008 and ITRF2014 have a plate motion model. </br>
Example: `'epoch': '2018.1'` (This specifies January 2018.)

**ITRF values associated with each mission:**

ICESat-2: `ITRF2014`

ICESat/Glas: `ITRF2008`
  
IceBridge/Pre-IceBridge ILATM1B: `ITRF2008`

IceBridge ILVIS2: `ITRF2000`

---
#### 2.3. Accessing Data with the *IceFlow* Access Widget
The *IceFlow* access widget is a user interface tool to visualize flightpaths from IceBridge, draw a region of interest, set spatio-temporal parameters and place data orders to the *IceFlow* API and *icepyx* package without the need to write code.
The output of the operations performed in the widget can be seen in the log window (right-most icon at the bottom left corner of your browser.) 
<img src='./img/log-icons.png'> or by selecting it on the _View_ menu "Show log console"

In [ ]:
# Let's start with the user interface. Using 'horizontal' will add the widget inline.
client.display_map('horizontal', extra_layers=True)

---
#### 2.4. Accessing data with the *IceFlow* API

IceFlow orders can also be placed programmatically outside of the map widget by manually selecting the data sets, temporal and spatial ranges, and optional georeferencing parameters. The number of files, or granules, found over the region of interest, as well as the original data volume prior to subsetting, is returned below.

#### Define order parameters and view data availability

In [ ]:
# Small example subset over Sermeq Kujalleq (Jakobshavn Isbrae):
my_params1 ={
    'datasets': ['GLAH06', 'ATL06'],
    'start': '2007-01-01',
    'end': '2018-12-31',
    'bbox': '-49.6,69.1,-49.3,69.17',
    # Here we will select ITRF2014 to match the Epoch of the most recent ICESat-2 granule we are ordering
    'itrf': 'ITRF2014',
    'epoch': '2018.12'
}

# returns a json dictionary, the request parameters, and the order's response.
granules_metadata = client.query_cmr(params=my_params1)

Only ICESat/GLAS and ICESat-2 data are included above due to the high density of ATM data available from IceBridge. Here we order a smaller subset of ATM data to decrease the order and download times. Again, the number of files and volume of ATM data from each IceBridge dataset is outputted below.

In [ ]:
# Since the IceBridge data is so dense, we order a smaller subset to decrease order and download times
my_params2 ={
    'datasets': ['ATM1B'],
    'start': '2007-01-01',
    'end': '2018-12-31',
    'bbox': '-49.53,69.12,-49.51,69.135',
    # Here we will select ITRF2014 to match the Epoch of the most recent ICESat-2 granule we are ordering
    'itrf': 'ITRF2014',
    'epoch': '2018.12'
}

# returns a json dictionary, the request parameters and the order's response.
granules_metadata = client.query_cmr(params=my_params2)

#### Order data

Now each order is placed. The order details are printed below: 

In [ ]:
orders1 = client.place_data_orders(params=my_params1)
print(orders1)
orders2 = client.place_data_orders(params=my_params2)
print(orders2)

#### Check Order Status
The following cell will show you the status of your data order. You can proceed in the notebook once all orders are "COMPLETE". If you proceed earlier only the completed data orders will be downloaded.

In [ ]:
for order in orders1:
    status = client.order_status(order)
    print(order['dataset'], order['id'], status['status'])
    
for order in orders2:
    status = client.order_status(order)
    print(order['dataset'], order['id'], status['status'])

#### Download Data
Once all data orders are "COMPLETE", you can proceed downloading the data. The data are downloaded to the /data folder of this notebook directory.

In [ ]:
for order in orders1:
    status = client.order_status(order)
    if status['status'] == 'COMPLETE':
        client.download_order(order)

In [ ]:
for order in orders2:
    status = client.order_status(order)
    if status['status'] == 'COMPLETE':
        client.download_order(order)

---
#### 2.5. Downloading ICESat-2 data [directly] with ***icepyx***
Behind the scenes, *IceFlow* is using the [*icepyx*](https://icepyx.readthedocs.io/en/latest/) Python package to download ICESat-2 data. *icepyx* is a standalone library that includes its own examples and documentation and welcomes contributions from data users (no previous GitHub or software development experience required!). Thus, it has a lot of additional functionality for querying, subsetting, ordering, and downloading ICESat-2 datasets (with in-the-works additions for data ingest into multiple formats), including making it easier to programmatically download data from multiple regions. 

Here we highlight some of the data visualization capabilities for exploring data prior to order and download.

In [ ]:
# Access the icepyx query object and import icepyx
# Note: if you would like to order additional ICESat-2 data using icepyx, you'll need to attach an Earthdata
# session to your icepyx query object (or re-login to Earthdata). See [icepyx examples](https://icepyx.readthedocs.io/en/latest/getting_started/example_link.html) for more details.
import icepyx as ipx
bbox_list = [float(val) for val in (my_params1["bbox"].split(","))]
is2_obj = ipx.Query(str(my_params1["datasets"][-1]), bbox_list, [my_params1["start"], my_params1["end"]])

We can visualize the extent of the query object created in the previous cell:

In [ ]:
# Visualize the query extent (this map won't be interactive if you don't have geoviews and the dev version of icepyx installed)
# Thus, for very small areas it can be difficult to see the specified region on a static world map (an area for future development!)
is2_obj.visualize_spatial_extent()

## 3. Working with the data
Now that we have downloaded our data, we need to make sure that they are in a common format to do analysis across missions.

Although typically we would include all import statements at the start of the workflow, here we have separated them into this section for instructional clarity.

The main Python packages/libraries that will be used in this notebook are:

* [*cartopy*](https://scitools.org.uk/cartopy/docs/latest/):
A Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.
* [*geopandas*](https://geopandas.org/): 
Library to simplify working with geospatial data in Python (using pandas).
* [*h5py*](https://github.com/h5py/h5py):
Pythonic wrapper around the [*HDF5 library](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) 
* [*matplotlib*](https://matplotlib.org/):
Comprehensive library for creating static, animated, and interactive visualizations in Python
* [*vaex*](https://github.com/vaexio/vaex):
High performance Python library for lazy Out-of-Core dataframes (similar to *pandas*), to visualize and explore big tabular data sets
* [*pandas*](https://pandas.pydata.org/):
Open source data analysis and manipulation tool
* [*icepyx*](https://icepyx.readthedocs.io/en/latest/):
Library for ICESat-2 data users
 
**Note**: *Warnings* are being ignored to suppress verbose warnings from some libraries (i.e. vaex, h5py). This will not prevent users from seeing errors.

In [ ]:
import cartopy.crs as ccrs #geospatial (mapping) plotting library
import cartopy.io.img_tiles as cimgt
import geopandas as gpd #add geospatial awareness/functionality to pandas
import h5py
from iceflow.processing import IceFlowProcessing as ifp
%matplotlib widget
import matplotlib.pyplot as plt #Python visualization
import vaex
import pandas as pd #data analysis and manipulation tool
import numpy as np
import warnings #Python warnings module
warnings.filterwarnings("ignore")

#### 3.1. Import data and convert to a geopandas data frame
ICESat, ICESat-2 and IceBridge data can be read in using preconfigured common dictionaries. Note that the conversion to a geopandas dataframe for the IceBridge ATM data is commented out below, since we have chosen to downsample those data in the next section.

**NOTE: Due to the size of the files explored in this tutorial, the data must be downloaded from section 2.4 instead of accessing them directly from the repository. The filenames will need to be updated in the following code blocks using the output filenames that you have downloaded from your orders.**

In [ ]:
# ICESat granule data
glas_gdf = ifp.to_geopandas('data/GLAH06-20210423-Sample.h5') # UPDATE PATH BASED ON YOUR OUTPUTTED FILENAME
glas_gdf['mission'] = "IS"

# #Pre-IceBridge/IceBridge ATM granule data
# preib_gdf = ifp.to_geopandas('data/ATM1B-20210423-Sample.h5') # UPDATE PATH BASED ON YOUR OUTPUTTED FILENAME
# preib_gdf['mission'] = "IB"

In [ ]:
# ICESat-2 granule data
is2_gdf = ifp.to_geopandas('data/ATL06-20181214041627-Sample.h5') # UPDATE PATH BASED ON YOUR OUTPUTTED FILENAME
is2_gdf['mission'] = "IS2"

Each geolocated dataframe has the same harmonized variables across each mission:

In [ ]:
# Let's see what's in the harmonized dataframe and its shape.
display(glas_gdf.head(), glas_gdf.shape)

In [ ]:
# and again for ICESat-2 ATL06
display(is2_gdf.head(), is2_gdf.shape)

#### 3.2. Downsample IceBridge Data
Due to the size of the IceBridge ATM1B point cloud, it is often difficult to work with or plot data in a Notebook environment. We will downsample the data in this example for faster plotting.

In [ ]:
# Read in the data and common dictionary
filepath = 'data/ATM1B-20210423-Sample.h5' # UPDATE PATH BASED ON YOUR OUTPUTTED FILENAME.
atm_key = ifp.get_common_dictionary('ATM')

f = h5py.File(filepath, 'r')
preib_vx = vaex.open(filepath)

preib_vx['time'] = preib_vx.utc_datetime.values.astype('datetime64[ns]')
preib_df = preib_vx[atm_key['latitude'], atm_key['longitude'], atm_key['elevation'], 'time']
preib_df.add_column('index', vaex.vrange(0, len(preib_vx)))
display(preib_df)

In [ ]:
# Here we will aggregate or "decimate" the data to make it smaller for our purposes
preib_dec = preib_df[(preib_df.index % 100 == 0)]
ib = np.array(["IB"]*len(preib_df))
preib_dec.add_column('mission', ib)
display(preib_dec)

In [ ]:
# Now we need to convert our downsampled data back into a pandas geodataframe so we can merge it with the other missions
preib_pandas = preib_dec.to_pandas_df(["latitude","longitude", "elevation", "time", "mission"])

preib_gdf = gpd.GeoDataFrame(preib_pandas,
                                geometry=gpd.points_from_xy(preib_pandas['longitude'],
                                                            preib_pandas['latitude'],
                                                            crs='epsg:4326'))
display(preib_gdf.head(), preib_gdf.shape)

#### 3.3. Plot the data from each mission together

Now, let's plot all three datasets together, using color to show the changes in elevation. 

In [ ]:
# Note that although this data is projected, it is not recommended you use this map as a basis for geospatial analysis

# Create a Stamen terrain background instance.
stamen_terrain = cimgt.Stamen('terrain-background')

map_fig = plt.figure()
# Create a GeoAxes in the tile's projection.
map_ax = map_fig.add_subplot(111, projection=stamen_terrain.crs)

# Limit the extent of the map to a small longitude/latitude range.
map_ax.set_extent([-56, -45, 67, 71], crs=ccrs.Geodetic())

# Add the Stamen data at zoom level 8.
map_ax.add_image(stamen_terrain, 8)

for onegdf, lab, shp in zip([preib_gdf, glas_gdf, is2_gdf],["preib","is", "is2"], ['P','o','D']):
    ms=map_ax.scatter(onegdf["longitude"], onegdf["latitude"],  2, c=onegdf["elevation"],
                      vmin=0, vmax=1000, label=lab, marker=shp,
                      transform=ccrs.Geodetic())

plt.colorbar(ms, label='elevation');

#### 3.4. Stack the dataframes

Thanks to the harmonization provided by IceFlow, we can stack our geopandas dataframes to have a unified dataframe for analysis.

In [ ]:
stacked_df = gpd.GeoDataFrame(pd.concat([preib_gdf, glas_gdf, is2_gdf]))
display(stacked_df.head(), stacked_df.shape)

#### 3.5. Time series analysis

Finally, we demonstrate the ability to do a time series across all three missions. We will zoom in on a very small section of our data to plot the time series.

In [ ]:
tsgdf_full = stacked_df.loc[(stacked_df["longitude"]>=-49.526) & (stacked_df["longitude"]<=-49.521)
              & (stacked_df["latitude"]>=69.121) & (stacked_df["latitude"]<=69.125) ]
# filter out erroneous elevation values
tsgdf_full = tsgdf_full.loc[tsgdf_full["elevation"] > 0]

print(len(tsgdf_full))

In [ ]:
# in order to plot as a time series, we cannot have duplicate x (time) values. Since the data collection rates
# are on the order of seconds, we keep the average where there are multiple records per second
tsgdf = tsgdf_full.groupby('time').mean()

# we also need to make "time" a non-index column
tsgdf["time_col"] = tsgdf.index

In [ ]:
tsgdf.plot(x="time_col",y="elevation", kind="scatter");

---